In [1]:
# Notebook cell: Stoichiometric surface energy (auto area axis + energy/atom)
# ---------------------------------------------------------------
import os, glob, json
from pathlib import Path
import numpy as np
import pandas as pd
from ase.io import read
from ase.atoms import Atoms

# ===================== USER CONFIG =====================
BASE = "/home/phanim/harshitrawat/summer/MS_LLZO_surface_data-master"
REF_BULK = os.path.join(BASE, "refs", "LLZO_tet_bulk.cif")
STOICH_DIR = os.path.join(BASE, "stoichiometric")
OUTPUT_XLSX = os.path.join(BASE, "surface_energy_stoich.xlsx")
OUTPUT_JSON = os.path.join(BASE, "surface_energy_stoich.json")

# Energy backend: "CHGNET", "MACE", "OUTCAR", or "EMT" (placeholder)
ENERGY_MODE = "CHGNET"

# If using MACE:
MACE_MODEL_PATH = "/path/to/your/mace.model"
MACE_DEVICE = "cuda:0"  # or "cpu"

# Bulk formula units for LLZO
LLZO_FORMULA = {"Li": 7, "La": 3, "Zr": 2, "O": 12}
LLZO_ELEMENTS = set(LLZO_FORMULA.keys())

EV_TO_J = 1.602176634e-19
A2_TO_M2 = 1e-20
# =======================================================

# ---------- Energy helpers ----------
def get_energy(at: Atoms, mode: str) -> float:
    """Return potential energy in eV for an ASE Atoms using the selected backend."""
    mode = mode.upper()
    if mode == "CHGNET":
        from chgnet.model.dynamics import CHGNetCalculator
        at.calc = CHGNetCalculator()
        return float(at.get_potential_energy())
    elif mode == "MACE":
        from mace.calculators import MACECalculator
        calc = MACECalculator(model_path=MACE_MODEL_PATH, device=MACE_DEVICE, head="target_head")
        at.calc = calc
        return float(at.get_potential_energy())
    elif mode == "OUTCAR":
        raise NotImplementedError("OUTCAR parsing not implemented in this snippet.")
    elif mode == "EMT":
        from ase.calculators.emt import EMT
        at.calc = EMT()
        return float(at.get_potential_energy())
    else:
        raise ValueError(f"Unknown ENERGY_MODE: {mode}")

# ---------- Composition helpers ----------
def count_elements(at: Atoms) -> dict:
    from collections import Counter
    return dict(Counter(at.get_chemical_symbols()))

def formula_units_in_cell(counts: dict, base_formula: dict) -> int:
    ratios = [(counts.get(el, 0) / req) for el, req in base_formula.items()]
    n = min(ratios)
    n_int = int(round(n))
    if not (abs(n - n_int) < 1e-6):
        raise ValueError(f"Non-integer f.u. detected (n={n}); slab may not be stoichiometric. Counts={counts}")
    return n_int

# ---------- Geometry / area detection ----------
def _axis_dir(cell, i):
    """Unit direction vector along lattice vector i."""
    v = np.array(cell[i], dtype=float)
    n = np.linalg.norm(v)
    return v / n, n

def atomic_span_along_axis(at: Atoms, axis_i: int):
    """
    Project atomic Cartesian positions onto lattice vector i (unit direction),
    return min, max, span (Å). Uses wrapped cartesian positions.
    """
    dir_i, length_i = _axis_dir(at.cell, axis_i)
    pos = at.get_positions()
    projs = pos @ dir_i  # projection lengths along axis direction
    return float(projs.min()), float(projs.max()), float(projs.max() - projs.min()), float(length_i)

def vacuum_fraction(at: Atoms, axis_i: int):
    """Estimate fraction of cell length along axis_i that is vacuum."""
    _, _, span, length = atomic_span_along_axis(at, axis_i)
    span = min(span, length)  # safety
    return max(0.0, 1.0 - (span / max(length, 1e-12)))

def auto_inplane_area(at: Atoms):
    """
    Auto-detect vacuum axis -> return:
      area_A2, vacuum_axis (0/1/2), inplane_axes (tuple), method ("pbc" or "vacuum_fraction")
    Strategy:
      1) If pbc is set and exactly one axis is non-periodic, that's vacuum.
      2) Else, pick axis with largest vacuum_fraction (tie -> largest length).
    """
    cell = at.get_cell()
    a, b, c = np.array(cell[0]), np.array(cell[1]), np.array(cell[2])
    pbc = at.get_pbc()

    # 1) pbc-based detection
    if isinstance(pbc, (list, tuple, np.ndarray)) and len(pbc) == 3:
        false_axes = [i for i, flag in enumerate(pbc) if not bool(flag)]
        if len(false_axes) == 1:
            vac = false_axes[0]
            inplane = [0, 1, 2]
            inplane.remove(vac)
            area = np.linalg.norm(np.cross(cell[inplane[0]], cell[inplane[1]]))
            return float(area), int(vac), tuple(inplane), "pbc"

    # 2) geometric heuristic on vacuum_fraction
    vf = [vacuum_fraction(at, i) for i in range(3)]
    lengths = [np.linalg.norm(cell[i]) for i in range(3)]
    # pick axis with maximum (vacuum_fraction, length)
    vac = int(np.lexsort((lengths, vf))[-1])  # highest vf, break ties by longest axis
    inplane = [0, 1, 2]
    inplane.remove(vac)
    area = np.linalg.norm(np.cross(cell[inplane[0]], cell[inplane[1]]))
    return float(area), vac, tuple(inplane), "vacuum_fraction"

def parse_facet_and_term(path: str):
    p = Path(path)
    facet, term = None, None
    for s in p.parts:
        if "_terminated_" in s:
            term = s.split("_terminated_")[0] + "_terminated"
            facet = s.split("_terminated_")[1]
            break
    return facet, term

# ---------- Bulk reference ----------
bulk = read(REF_BULK)
bulk_counts = count_elements(bulk)
bulk_fu = formula_units_in_cell(bulk_counts, LLZO_FORMULA)

E_bulk = get_energy(bulk, ENERGY_MODE)  # eV
E_bulk_per_fu = E_bulk / bulk_fu
E_bulk_per_atom = E_bulk / len(bulk)

print(f"Bulk reference: {REF_BULK}")
print(f"Bulk E_total = {E_bulk:.6f} eV over {bulk_fu} f.u. => {E_bulk_per_fu:.6f} eV/f.u.; {E_bulk_per_atom:.6f} eV/atom")

# ---------- Iterate stoichiometric slabs ----------
rows = []
vasp_files = sorted(glob.glob(os.path.join(STOICH_DIR, "**", "*.vasp"), recursive=True))

for fpath in vasp_files:
    slab = read(fpath)
    counts = count_elements(slab)

    # strictly stoichiometric species check
    if set(counts) - LLZO_ELEMENTS:
        continue

    n_fu = formula_units_in_cell(counts, LLZO_FORMULA)
    n_atoms = len(slab)
    E_slab = get_energy(slab, ENERGY_MODE)  # eV
    E_slab_per_atom = E_slab / n_atoms

    area_A2, vac_axis, inplane_axes, method = auto_inplane_area(slab)
    if area_A2 <= 0.0:
        raise ValueError(f"Non-positive in-plane area for {fpath}")

    gamma_eV_A2 = (E_slab - n_fu * E_bulk_per_fu) / (2.0 * area_A2)
    gamma_J_m2  = gamma_eV_A2 * (EV_TO_J / A2_TO_M2)

    facet, term = parse_facet_and_term(fpath)
    rows.append({
        "file": fpath,
        "facet": facet,
        "termination": term,
        "n_formula_units": n_fu,
        "n_atoms": n_atoms,
        "vacuum_axis": vac_axis,                # 0=a, 1=b, 2=c
        "inplane_axes": f"{inplane_axes}",      # tuple as string for Excel
        "area_detection": method,
        "area_A2": area_A2,
        "E_slab_eV": E_slab,
        "E_slab_eV_per_atom": E_slab_per_atom,
        "E_bulk_per_fu_eV": E_bulk_per_fu,
        "E_bulk_eV_per_atom": E_bulk_per_atom,
        "gamma_eV_A2": gamma_eV_A2,
        "gamma_J_m2": gamma_J_m2,
    })

df = pd.DataFrame(rows).sort_values(
    ["facet", "termination", "gamma_J_m2", "file"], na_position="last"
).reset_index(drop=True)

# Save
df.to_excel(OUTPUT_XLSX, index=False)
with open(OUTPUT_JSON, "w") as f:
    json.dump(df.to_dict(orient="records"), f, indent=2)

print(f"\nWrote {len(df)} rows")
print(f"Excel: {OUTPUT_XLSX}")
print(f"JSON : {OUTPUT_JSON}")
df.head()


/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/python3.10/site-packages/chgnet/model/model.py:673: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cpu
Bulk reference: /home/phanim/harshitrawat/summer/MS_LLZO_surface_data-master/refs/LLZO_tet_bulk.cif
Bulk E_total = -1429.347015 eV over 8 f.u. => -178.668377 eV/f.u.; -7.444516 eV/atom
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cpu


KeyboardInterrupt: 